In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[38]:


import numpy as np
import pandas as pd
import re 
import matplotlib.pyplot as plt 
import sys


#Input file must be given to run the code.
#For Example Run(in terminal) 'python3 Bioinformatics input.xlsx'

#Filename =sys.argv[1] # Read input file 

df = pd.read_excel('input.xlsx')# Read EXCEL Files and storing in database 'df'
data= pd.DataFrame()# Creating New Data Frame 


# Storing Usefull information From the database 'df' 
data['Entry']=df['Entry'] #Storing Entry
data['Entry name']=df['Entry name']  #Storing Entry Name
data['Protein names']=df['Protein names']  #Storing Protein Names
data['Gene names']=df['Gene names'] # Storing Gene Names
data['Length']=df['Length'] # Storing Length
data['Organism']=df['Organism'] # Storing Type of Organism
data['Mass']=df['Mass'] #Storing Mass
data['Status']=df['Status']

#Now removing the extra data we dont need and containg only organism 'Human'
columns=data[data['Organism'].str.contains("Human")] #Selecting only organism which contains HUMANS
data= columns

data['Disulfide bond']=df['Disulfide bond'] # Storing Disulfide bond 
data['Glycosylation']= df['Glycosylation']# Storing Glycosylation Position
data=data.dropna().reset_index(drop=True) # Drop all rows which contains Not a number and reset the index 

# Working on getting the relative positions of the Disulfide bond
disulfide_column= data['Disulfide bond'] # Storing data temporary as 'disulfide_column'
def getdata(newdata):
    return re.findall('\d+ \d+',newdata)

data['Disulfide bond']= disulfide_column.apply(getdata) # Function call which gives all the positons 



In [2]:

#Working on getting the positions of N-linked Glycosylation
Glyco_data= data['Glycosylation'] # Storing data temporary as 'Glyco_data'

def getGlycoNLinked(data):   #Making a Function to get the positions
    return re.findall('\d+\s* N-linked',data) # The return data will contain 'position and N-linked pattern'
data['Glycosylation']= Glyco_data.apply(getGlycoNLinked) # the return data will be like '49 N-linked'


#Now Removing the extra word 'N-linked' and getting all positions 
temp_data= data['Glycosylation'].astype(str)
def getValue(data):
    return re.findall('\d+',data) # return all positions
data['Glycosylation']= temp_data.apply(getValue) 


# Creating a temporary dataframe for total length in columns and length in rows(like number of pairs in each index).
a= pd.DataFrame()
b= pd.DataFrame()
bond=pd.DataFrame()
a =pd.Series(data['Glycosylation'])
b =pd.Series(data['Disulfide bond'])
bond=pd.DataFrame()
bond['data']=data['Disulfide bond']
lenB=b.str.len()
lenA=a.str.len()

rel_pos = pd.Series(a.size,dtype=np.str)
for i in range(a.size):
    rel_pos[i]=''
    
InsidePairs=pd.Series(b.size,dtype=np.str)
InsidePairsLength=np.zeros(b.size,dtype=np.int)
for i in range(a.size):
    InsidePairs[i]=''

# Now using loop and properties of Python library 're' to get values and finding positions between disulphide bond and glycosation.

i=0
while i<b.size:
    for j in range(lenA[i]):
        Position=int(a[i][j])
        rel_pos[i]=rel_pos[i]+str(a[i][j])+'{|'
        for k in range(lenB[i]):
            split = re.findall('\d+',b[i][k]) 
            firstPair= int(split[0])
            secondPair=int(split[1])
            if Position< firstPair:
                rel_pos[i] = rel_pos[i]+'o'+str(Position-firstPair)
                rel_pos[i] = rel_pos[i]+'o'+str(Position-secondPair)
            elif Position>= firstPair and Position<=secondPair:
                rel_pos[i]=  rel_pos[i]+'i' + str(Position-firstPair)
                rel_pos[i] = rel_pos[i]+'i'+ str(Position-secondPair)
                InsidePairs[i]= InsidePairs[i] + str('('+ str(firstPair)+','+str(secondPair)+')')
                InsidePairsLength[i]=InsidePairsLength[i]+ (secondPair-firstPair)
            else:
                rel_pos[i] = rel_pos[i]+'o'+str(Position-firstPair)
                rel_pos[i] = rel_pos[i]+'o'+str(Position-secondPair)
            rel_pos[i]=rel_pos[i]+'|'    
        rel_pos[i]=rel_pos[i]+'}'
    i=i+1
data['rel_pos']= rel_pos
data['In_Gly_Dis_Pair']=InsidePairs

In [3]:
#Calculating Interbond Distance
interbond_distance= pd.Series(a.size,dtype= np.str)

for i in range(a.size):
    interbond_distance[i]=''

for i in range(a.size):
    
    if lenB[i]==0:
        interbond_distance[i]='No pair'
        
    if lenB[i]==1:
        interbond_distance[i]='Only One pair'
    
    for r in range(lenB[i]-1):
        split1= re.findall('\d+',data['Disulfide bond'][i][r])
        split2= re.findall('\d+',data['Disulfide bond'][i][r+1])
        interbond_distance[i]= interbond_distance[i]+ str(int(split2[0])-int(split1[1]))+ '|'
        
data['interbond_distance']=interbond_distance

#Intrabond distance formation
intrabond=pd.Series(b.size,dtype=np.str)
for i in range(a.size):
    intrabond[i]=''
j=0;
while j<b.size:
    for k in range(lenB[j]):
        split = re.findall('\d+',b[j][k]) 
        firstPair= int(split[0])
        secondPair=int(split[1])
        intrabond[j]=intrabond[j]+str(secondPair-firstPair)+'|'    
    j=j+1
data['intrabond']=intrabond


#number of pairs of disuphide bonds
data['No. Disulphide bonds']=lenB


#1st disulphide pair to N-terminus distance
first_pos = pd.Series(a.size,dtype=np.int)
for r in range(a.size): 
    split = re.findall('\d+',data['Disulfide bond'][r][0]) 
    firstPair= int(split[0])
    first_pos[r]= firstPair
data['first_position_occurance']=first_pos


#last disulphide pair to C-terminus distance
last_pos= pd.Series(a.size,dtype=np.int)
for r in range(a.size):
    split= re.findall('\d+',data['Disulfide bond'][r][(lenB[r]-1)])
    lastPair= int(split[1])
    last_pos[r] = lastPair 
data['last_position_occurance']=last_pos

#Calculate average distance of intrabond distance
def intrabond_Average(data):
    return re.findall('\d+',data)
average= data['intrabond'].apply(intrabond_Average)
lenIntra=average.str.len()

averageintrabond= np.zeros(a.size)
for i in range(a.size):
    totalsum=0
    for j in range(lenIntra[i]):
        totalsum= totalsum+int(average[i][j])
    averageintrabond[i]=totalsum/lenIntra[i]
data['average_Intrabond']=averageintrabond

In [4]:
#calculating average distance of those pairs of disulphide bonds that have Glycosylation Inside.
temp_length=pd.Series(b.size,dtype=np.str)
for i in range(a.size):
    temp_length[i]=''
inside_length = np.zeros(a.size,dtype=np.int)
for r in range(a.size):
    inside_length[r]=len(re.findall('\d+',data['In_Gly_Dis_Pair'][r]))/2
    temp_length[r]=re.findall('\d+',data['In_Gly_Dis_Pair'][r])
    if inside_length[r]!=0:
        InsidePairsLength[r]=InsidePairsLength[r]/inside_length[r]
data['avg_Inside_Pairs_Length']=InsidePairsLength

In [5]:
#If the glyco is inside then putting the sulphide position 'nil' at that point.
i=0
while i<b.size:
    for j in range(lenA[i]):
        Position=int(a[i][j])
        for k in range(lenB[i]):
            if b[i][k]!='nil':
                split = re.findall('\d+',b[i][k]) 
                firstPair= int(split[0])
                secondPair=int(split[1])
                if Position>= firstPair and Position<=secondPair:
                    bond['data'][i][k]='nil'
    i=i+1

In [6]:
#calculating length between sulfide bonds.
temp=pd.Series(b.size,dtype=np.str)
for i in range(a.size):
    temp[i]=''

for i in range(len(bond)):
    for j in range(lenB[i]):
        if bond['data'][i][j]!='nil':
            split= re.findall('\d+',bond['data'][i][j])
            temp[i]=str(temp[i] + str(int(split[1])-int(split[0]))+'|')
data['glyco_outside_bond']=bond['data']

In [9]:
#get the integer values.
def getvalues(temp):
    return re.findall('\d+',temp)
intrabond_outside= temp.apply(getvalues)
data['intrabond_glyco_outside_bond']=intrabond_outside

In [23]:
#calculating avegrage lenth of sulphide bonds which do not have glyco inside.
new =pd.Series(intrabond_outside)
newlen=new.str.len()
average= np.zeros(a.size)
for i in range(a.size):
    totalsum=0
    for j in range(newlen[i]):
        totalsum= totalsum+int(intrabond_outside[i][j])
    if newlen[i]!=0:
        average[i]=totalsum/newlen[i]
data['average_Intrabond_outside_glyco_bond']=average

In [11]:
data

,Entry,Entry name,Protein names,Gene names,Length,Organism,Mass,Status,Disulfide bond,Glycosylation,...,interbond_distance,intrabond,No. Disulphide bonds,first_position_occurance,last_position_occurance,average_Intrabond,avg_Inside_Pairs_Length,glyco_outside_bond,intrabond_glyco_outside_bond,average_Intrabond_outside_glyco_bond
0,P04439,HLAA_HUMAN,"HLA class I histocompatibility antigen, A alph...",HLA-A HLAA,365,Homo sapiens (Human),"40,841",reviewed,"[125 188, 227 283]",[110],...,39|,63|56|,2,125,283,59.500000,0,"[125 188, 227 283]","[63, 56]",59.500000
1,P08246,ELNE_HUMAN,Neutrophil elastase (EC 3.4.21.37) (Bone marro...,ELANE ELA2,267,Homo sapiens (Human),"28,518",reviewed,"[55 71, nil, 181 187, 198 223]","[88, 124, 173]",...,80|-27|11|,16|57|6|25|,4,55,223,26.000000,57,"[55 71, nil, 181 187, 198 223]","[16, 6, 25]",15.666667
2,P10144,GRAB_HUMAN,Granzyme B (EC 3.4.21.79) (C11) (CTLA-1) (Cath...,GZMB CGL1 CSPB CTLA1 GRB,247,Homo sapiens (Human),"27,716",reviewed,"[49 65, 142 209, 173 188]","[71, 104]",...,77|-36|,16|67|15|,3,49,188,32.666667,0,"[49 65, 142 209, 173 188]","[16, 67, 15]",32.666667
3,P10321,HLAC_HUMAN,"HLA class I histocompatibility antigen, C alph...",HLA-C HLAC,366,Homo sapiens (Human),"40,649",reviewed,"[125 188, 227 283]",[110],...,39|,63|56|,2,125,283,59.500000,0,"[125 188, 227 283]","[63, 56]",59.500000
4,P01889,HLAB_HUMAN,"HLA class I histocompatibility antigen, B alph...",HLA-B HLAB,362,Homo sapiens (Human),"40,460",reviewed,"[125 188, 227 283]",[110],...,39|,63|56|,2,125,283,59.500000,0,"[125 188, 227 283]","[63, 56]",59.500000
5,P61916,NPC2_HUMAN,NPC intracellular cholesterol transporter 2 (E...,NPC2 HE1,151,Homo sapiens (Human),"16,570",reviewed,"[nil, 42 47, 93 99]","[58, 135]",...,-98|46|,113|5|6|,3,27,99,41.333333,113,"[nil, 42 47, 93 99]","[5, 6]",5.500000
6,Q6FHJ7,SFRP4_HUMAN,Secreted frizzled-related protein 4 (sFRP-4) (...,SFRP4 FRPHE,346,Homo sapiens (Human),"39,827",reviewed,"[nil, nil, 69 108, nil, nil]","[38, 68, 116, 194, 240]",...,-53|-9|-11|-35|,61|46|39|39|24|,5,24,125,41.800000,46,"[nil, nil, 69 108, nil, nil]",[39],39.000000
7,P22897,MRC1_HUMAN,Macrophage mannose receptor 1 (MMR) (C-type le...,MRC1 CLEC13D CLEC13DL MRC1L1,1456,Homo sapiens (Human),"166,012",reviewed,"[35 49, 74 91, 168 194, 182 209, 247 340, 316 ...","[104, 344, 529, 926, 930, 1160, 1205]",...,25|77|-12|38|-24|59|-23|54|-25|29|21|-24|59|-2...,14|17|26|27|93|16|95|15|93|17|13|97|16|94|15|1...,21,35,1347,46.619048,89,"[35 49, 74 91, 168 194, 182 209, 247 340, 316 ...","[14, 17, 26, 27, 93, 16, 95, 15, 93, 17, 13, 9...",44.500000
8,O75326,SEM7A_HUMAN,Semaphorin-7A (CDw108) (JMH blood group antige...,SEMA7A CD108 SEMAL,666,Homo sapiens (Human),"74,824",reviewed,"[120 126, 143 152, nil, nil, 493 511, 500 541,...","[105, 157, 258, 330, 602]",...,17|114|-75|158|-11|-38|48|-26|,6|9|100|44|18|41|15|47|9|,9,120,596,32.111111,63,"[120 126, 143 152, nil, nil, 493 511, 500 541,...","[6, 9, 18, 41, 15, 9]",16.333333
9,P78410,BT3A2_HUMAN,Butyrophilin subfamily 3 member A2,BTN3A2 BT3.2 BTF3 BTF4,334,Homo sapiens (Human),"36,428",reviewed,"[nil, 166 220]",[115],...,40|,74|54|,2,52,220,64.000000,74,"[nil, 166 220]",[54],54.000000


In [12]:
data.to_excel('output.xlsx')

In [21]:
newlen[newlen==0]

11      0
20      0
41      0
42      0
47      0
62      0
65      0
67      0
68      0
72      0
75      0
87      0
90      0
93      0
102     0
105     0
107     0
124     0
125     0
128     0
130     0
133     0
136     0
152     0
174     0
180     0
185     0
192     0
193     0
194     0
       ..
2758    0
2765    0
2766    0
2767    0
2779    0
2781    0
2796    0
2800    0
2809    0
2814    0
2819    0
2825    0
2843    0
2873    0
2882    0
2887    0
2888    0
2892    0
2893    0
2895    0
2896    0
2906    0
2908    0
2920    0
2932    0
2935    0
2948    0
2949    0
2950    0
2964    0
Length: 450, dtype: int64